##**Import Libraries**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report
# from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from yellowbrick.text import FreqDistVisualizer
from mlxtend.feature_selection import ColumnSelector

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

##**Import Data**

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Spam Msg/spam.csv',encoding = 'latin-1')

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
data.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [ ]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


##**EDA**

In [ ]:
data['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

##Preprocessing

---
Applying NLP protocols so as to clean text


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
en_stopwords = stopwords.words('english')
en_stopwords+=['one', 'two', 'go','goes', 'get', 'also', 'however', 'tells',''''s''']
en_stopwords += list(string.punctuation)

# Initialize WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text(text):
    text=text.lower()   #Lowercasing all text
    text=word_tokenize(text)  #Tokenization
    text=[w for w in text if w not in en_stopwords]  #Stopwords Filtering
    text = [lemmatizer.lemmatize(w) for w in text]
    text=' '.join(text)  #Reform sentence
    return text

In [ ]:
sample_text=data['v2'][5]
sample_text

"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, å£1.50 to rcv"

In [ ]:
clean_text(sample_text)

"freemsg hey darling 3 week word back 'd like fun still tb ok xxx std chgs send å£1.50 rcv"

In [ ]:
data['clean_text']=data['v2'].map(lambda x:clean_text(x))

In [ ]:
data['v1'].map({'ham':0,'spam':1})

0       0
1       0
2       1
3       0
4       0
       ..
5567    1
5568    0
5569    0
5570    0
5571    0
Name: v1, Length: 5572, dtype: int64

##**Splitting Data**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(data['clean_text'].values,data['v1'].values,test_size=0.25,random_state=123,stratify=data['v1'].values)

##**Vectorising Text so as to put as input to Algos**

###**1. Count Vactorizer**

In [ ]:
countvec = CountVectorizer(ngram_range = (1,3), max_features = 30000)
vec_Xtr = countvec.fit_transform(X_train)

In [ ]:
vec_Xte=countvec.fit_transform(X_test)

###**2. TF-IDF**

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)
tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

##**Training Models**

###1. Dummy Grid Classifier


---
The Dummy Grid Classifier will give us a base minimun accuracy for this project!

---
I will be using dummy grid so as toi find the best score


In [ ]:
dummy_grid = {
    'strategy': ['most_frequent', 'stratified', 'prior', 'uniform']
}
dummy = DummyClassifier( random_state = 42)
dummy_gs = GridSearchCV(dummy, param_grid = dummy_grid, cv = 5)
dummy_gs.fit(X_train, y_train)
print(dummy_gs.best_score_)

0.8659966192017878


###Base Accuracy: 0.8659966192017878

###**1. Naive Bias - TF-IDF**

In [ ]:
mnb_grid = {
    'alpha': [0, 0.2, 0.5, 0.8, 1.0, 1.0],
    'fit_prior' : [True, False],
    'class_prior' : [None, [.4, .3, 0.08, .07, .06, .06], [.167, .167, .167, .167, .167, .167]]
}
mnb = MultinomialNB()
mnb_tfidf_gs = GridSearchCV(mnb, param_grid = mnb_grid, cv = 5)
mnb_tfidf_gs.fit(tfidf_train_vectors, y_train)
print(mnb_tfidf_gs.best_score_)
predicted_NB=mnb_tfidf_gs.predict(tfidf_test_vectors)
print('\nClassification report:\n', classification_report(y_test, predicted_NB))
conf_mat = confusion_matrix(y_true=y_test, y_pred=predicted_NB)
print('Confusion matrix:\n', conf_mat)

0.984204509641005

Classification report:
               precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1206
        spam       0.99      0.94      0.96       187

    accuracy                           0.99      1393
   macro avg       0.99      0.97      0.98      1393
weighted avg       0.99      0.99      0.99      1393

Confusion matrix:
 [[1205    1]
 [  12  175]]


###**2. Naive Bias - Count Vectoriser**

In [ ]:
mnb_grid = {
    'alpha': [0, 0.2, 0.5, 0.8, 1.0],
    'fit_prior' : [True, False],
    'class_prior' : [None, [.4, .3, 0.08, .07, .06, .06], [.167, .167, .167, .167, .167, .167]]
}
mnb1 = MultinomialNB()
mnb_cvec_gs = GridSearchCV(mnb1, param_grid = mnb_grid, cv = 5)
mnb_cvec_gs.fit(vec_Xtr_new, y_train)
print(mnb_cvec_gs.best_score_)
predicted_NB_CV=mnb_cvec_gs.predict(vec_Xte_new)
print('\nClassification report:\n', classification_report(y_test, predicted_NB_CV))
conf_mat = confusion_matrix(y_true=y_test, y_pred=predicted_NB_CV)
print('Confusion matrix:\n', conf_mat)

0.9822914935678881

Classification report:
               precision    recall  f1-score   support

         ham       1.00      0.97      0.98      1206
        spam       0.83      0.97      0.89       187

    accuracy                           0.97      1393
   macro avg       0.91      0.97      0.94      1393
weighted avg       0.97      0.97      0.97      1393

Confusion matrix:
 [[1168   38]
 [   5  182]]


###**3. Logistic Regression - TF-IDF**

In [ ]:
lr_grid = {
    'penalty' : ['l1', 'l2', 'none'],
    'C' : [1.0, 1e2, 1e4, 1e10],
    'class_weight' : ['balanced', None],
    'solver' : ['lbfgs', 'liblinear'],
    'multi_class' : ['ovr', 'multinomial']
}
logreg = LogisticRegression(max_iter=1000,random_state = 42)

logreg_tfidf_gs = GridSearchCV(logreg, param_grid = lr_grid, cv = 5)
logreg_tfidf_gs.fit(tfidf_train_vectors, y_train)
print(logreg_tfidf_gs.best_score_)


0.9789419247629143


###**4. Logistic Regression - Count Vectoriser**

In [ ]:
logreg1 = LogisticRegression(max_iter=1000,random_state = 42)

logreg_cv = GridSearchCV(logreg, param_grid = lr_grid, cv = 5)
logreg_cv.fit(vec_Xtr, y_train)


0.9789419247629143


###5**. Decision tree - TF-IDF**

In [ ]:
grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 5, 10],
    'min_samples_split': [1, 5, 10, 20]
}

dt_clf = DecisionTreeClassifier (random_state = 42)
dt_tfidf = GridSearchCV(dt_clf, param_grid = grid, cv = 5)
dt_tfidf.fit(tfidf_train_vectors, y_train)

print(dt_tfidf.best_score_)

0.9614717932555941


###6**. Decision tree - Count Vectoriser**

In [ ]:
grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 2, 5, 10],
    'min_samples_split': [1, 5, 10, 20]
}
dt_clf1 = DecisionTreeClassifier (random_state = 42)
dt_cv = GridSearchCV(dt_clf1, param_grid = grid, cv = 5)
dt_cv.fit(vec_Xtr, y_train)

print(dt_cv.best_score_)

0.9554917915365442


###**7. SVM - TF-IDF**

In [ ]:
from sklearn.svm import SVC
svc_classifier = SVC(kernel='linear')
svc_classifier.fit(tfidf_train_vectors, y_train)


# Make Prediction & print the result
predicted_SVM_tfidf= svc_classifier.predict(tfidf_test_vectors)
print(classification_report(y_test,predicted_SVM_tfidf))
conf_mat_SVM = confusion_matrix(y_true=y_test, y_pred=predicted_SVM_tfidf)


              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1206
        spam       0.98      0.93      0.95       187

    accuracy                           0.99      1393
   macro avg       0.98      0.96      0.97      1393
weighted avg       0.99      0.99      0.99      1393

Confusion matrix:
 [[1205    1]
 [  12  175]]


In [ ]:
vec_Xte_new=countvec.transform(X_test)
vec_Xtr_new=countvec.transform(X_train)

###**8. SVM - Count Vectoriser**

In [ ]:
svc_classifier1 = SVC(kernel='linear')
svc_classifier1.fit(vec_Xtr_new, y_train)


# Make Prediction & print the result
predicted_SVM_cv= svc_classifier1.predict(vec_Xte_new)
print(classification_report(y_test,predicted_SVM_cv))
conf_mat_SVM = confusion_matrix(y_true=y_test, y_pred=predicted_SVM_cv)
print('Confusion matrix:\n', conf_mat_SVM)

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99      1206
        spam       0.95      0.94      0.94       187

    accuracy                           0.98      1393
   macro avg       0.97      0.96      0.97      1393
weighted avg       0.98      0.98      0.98      1393

Confusion matrix:
 [[1196   10]
 [  12  175]]
